In [66]:
# import pandas lib as pd
import pandas as pd
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# read by default 1st sheet of an excel file
# dataframe1 = pd.read_excel('E://REVA//Healthcare Providers Data For Anomaly Detection//Medicare1//2021_CA_Data.xlsx',sheet_name="Sheet1",header=0)
dataframe1 = pd.read_excel('E://GitHub//healthcare_provider_recommendation//2021_CA_Data.xlsx',sheet_name="data1",header=0)

In [67]:
df1 = dataframe1[dataframe1['Rndrng_Prvdr_Zip5'] == 90095] #user input
df2=df1[df1["HCPCS_Desc"].str.contains("patient")]
df2 = df2[df2['HCPCS_Cd'] == 99205] #user input
df3=df2
# Export the TF-IDF DataFrame to an Excel file
# df3.to_excel('E://REVA//Healthcare Providers Data For Anomaly Detection//Medicare1//df3_output.xlsx', index=False, engine='openpyxl')

In [68]:
# df3.loc[:, 'Avg_Sbmtd_Chrg_rank_default'] = df3['Avg_Sbmtd_Chrg'].rank(ascending=False, method='min').astype(int)
df3.loc[:, 'Avg_Sbmtd_Chrg_rank_natop'] = df3['Avg_Sbmtd_Chrg'].rank(ascending=True, method='min').astype(int)
# df3.loc[:, 'Avg_Sbmtd_Chrg_rank_natop'] = df3['Avg_Sbmtd_Chrg'].rank(na_option='top')
# df3.loc[:, 'Avg_Sbmtd_Chrg_rank_nabottom'] = df3['Avg_Sbmtd_Chrg'].rank(na_option='bottom')


# df3.loc[:, 'Tot_Bene_Day_Srvcs_rank_default'] = df3['Tot_Bene_Day_Srvcs'].rank()
# df3.loc[:, 'Tot_Bene_Day_Srvcs_rank_natop'] = df3['Tot_Bene_Day_Srvcs'].rank(na_option='top')
# df3.loc[:, 'Tot_Bene_Day_Srvcs_rank_natop'] = df3['Tot_Bene_Day_Srvcs'].rank(ascending=True, method='max').astype(int)
# df3.loc[:, 'Tot_Bene_Day_Srvcs_rank_natop2'] = df3['Tot_Bene_Day_Srvcs'].rank(ascending=True, method='min').astype(int)
df3.loc[:, 'Tot_Bene_Day_Srvcs_rank_natop'] = df3['Tot_Bene_Day_Srvcs'].rank(ascending=True, method='min').astype(int)

# Find the maximum rank value in the 'Tot_Bene_Day_Srvcs_rank_natop2' column
max_rank_value = df3['Tot_Bene_Day_Srvcs_rank_natop'].max()

# Create a new column with values in reverse order
df3['Reverse_Rank_Tot_Bene_Day_Srvcs_rank_natop'] = max_rank_value - df3['Tot_Bene_Day_Srvcs_rank_natop'] + 1

# df3.loc[:, 'Tot_Bene_Day_Srvcs_rank_nabottom'] = df3['Tot_Bene_Day_Srvcs'].rank(na_option='bottom')
df3.to_excel('E://REVA//Healthcare Providers Data For Anomaly Detection//Medicare1//df3_06_output.xlsx', index=False, engine='openpyxl')

In [70]:
Tot_Bene_Day_Srvcs_percentile_95 = df4['Tot_Bene_Day_Srvcs'].quantile(0.95)
Avg_Sbmtd_Chrg_bottom_percentile_95 = df4['Avg_Sbmtd_Chrg'].quantile(0.5)


# Function to determine the quality tier
def determine_tier(row):
    if row['Tot_Bene_Day_Srvcs'] >= Tot_Bene_Day_Srvcs_percentile_95 :
        return 1
    else:
        return 2

# Apply the function to create the 'Quality Tier' column
df4['Quality_Tier_Bene_Day_Srvcs'] = df4.apply(determine_tier, axis=1)

# Function to determine the quality tier
def determine_tier(row):
    if  row['Avg_Sbmtd_Chrg'] <= Avg_Sbmtd_Chrg_bottom_percentile_95:
        return 1
    else:
        return 2

# Apply the function to create the 'Quality Tier' column
df4['Quality_Tier_Avg_Sbmtd_Chrg'] = df4.apply(determine_tier, axis=1)

df4['Quality_Tier2'] = (df4['Quality_Tier_Bene_Day_Srvcs'] + df4['Quality_Tier_Avg_Sbmtd_Chrg']) / 2


# df3.loc[:, 'Tot_Bene_Day_Srvcs_rank_nabottom'] = df3['Tot_Bene_Day_Srvcs'].rank(na_option='bottom')
df4.to_excel('E://REVA//Healthcare Providers Data For Anomaly Detection//Medicare1//df4_02_output.xlsx', index=False, engine='openpyxl')

In [36]:
# Group by 'Provider_Name' and calculate the sum of 'Tot_Bene_Day_Srvcs'
result = df4.groupby('Rndrng_NPI')['Tot_Bene_Day_Srvcs'].sum().reset_index()
result.columns = ['Rndrng_NPI', 'Tot_Bene_Day_across_all_Services_by_Provider']

# Merge the result DataFrame back to the original DataFrame
df6 = df4.merge(result, on='Rndrng_NPI', how='left')

# Multiply 'Field1' and 'Field2' to create a new 'Result' column
df6['Total_Charge_Claimed_By_Provider'] = df6['Tot_Bene_Day_Srvcs'] * df6['Avg_Sbmtd_Chrg']

# Group by 'Provider_Name' and calculate the sum of ('Tot_Bene_Day_Srvcs' * 'Avg_Sbmtd_Chrg')
result = df6.groupby('Rndrng_NPI')['Total_Charge_Claimed_By_Provider'].sum().reset_index()
result.columns = ['Rndrng_NPI', 'Total_Charge_Claimed_By_Provider']

# Merge the result DataFrame back to the original DataFrame
df7 = df6.merge(result, on='Rndrng_NPI', how='left')

# Group by 'Provider_Name' and calculate the sum of ('Tot_Bene_Day_Srvcs' * 'Avg_Sbmtd_Chrg')
result = df6.groupby('Rndrng_NPI')['Total_Charge_Claimed_By_Provider'].mean().reset_index()
result.columns = ['Rndrng_NPI', 'Average_Mean_Charge_Claimed_Across_All_Services_By_Provider']

# Merge the result DataFrame back to the original DataFrame
df7 = df6.merge(result, on='Rndrng_NPI', how='left')

In [37]:
df7.head()

Rndrng_NPI Rndrng_Prvdr_Last_Org_Name Rndrng_Prvdr_First_Name  \
0  1013113679                   Weingrow                  Daniel   
1  1013335355                    Emeruwa               Iheanacho   
2  1023288719                      Sayah                   David   
3  1023345048                   Juillard               Catherine   
4  1023456118                      Wolfe                 Michael   

  Rndrng_Prvdr_MI Rndrng_Prvdr_Crdntls Rndrng_Prvdr_Gndr Rndrng_Prvdr_Ent_Cd  \
0             NaN                  NaN                 M                   I   
1               O              MD, MBA                 M                   I   
2               M              MD, PHD                 M                   I   
3               J                 M.D.                 F                   I   
4               W                   MD                 M                   I   

                     Rndrng_Prvdr_St1 Rndrng_Prvdr_St2 Rndrng_Prvdr_City  ...  \
0                    757 Westwood Plz              NaN       Los Angeles  ...   
1       200 Ucla Medical Plz Ste 365b              NaN       Los Angeles  ...   
2  200 Ucla Medical Plaza, Ste. 365-B       Box 951693       Los Angeles  ...   
3        200 Ucla Medical Plz Ste 214              NaN       Los Angeles  ...   
4               757 Westwood Plz # Cu              NaN       Los Angeles  ...   

  Avg_Sbmtd_Chrg_rank_default  Avg_Sbmtd_Chrg_rank_natop  \
0                      3741.5                     3741.5   
1                      3741.5                     3741.5   
2                      3741.5                     3741.5   
3                      3741.5                     3741.5   
4                      3741.5                     3741.5   

   Avg_Sbmtd_Chrg_rank_nabottom  Tot_Bene_Day_Srvcs_rank_default  \
0                        3741.5                           2877.5   
1                        3741.5                           2805.0   
2                        3741.5                           3146.0   
3                        3741.5                           1352.0   
4                        3741.5                           2703.0   

  Tot_Bene_Day_Srvcs_rank_natop Tot_Bene_Day_Srvcs_rank_nabottom Quality Tier  \
0                        2877.5                           2877.5            2   
1                        2805.0                           2805.0            2   
2                        3146.0                           3146.0            2   
3                        1352.0                           1352.0            2   
4                        2703.0                           2703.0            2   

  Tot_Bene_Day_across_all_Services_by_Provider  \
0                                          103   
1                                           96   
2                                          143   
3                                           28   
4                                           86   

  Total_Charge_Claimed_By_Provider  \
0                         185400.0   
1                         172800.0   
2                         257400.0   
3                          50400.0   
4                         154800.0   

  Average_Mean_Charge_Claimed_Across_All_Services_By_Provider  
0                                           185400.0           
1                                           172800.0           
2                                           257400.0           
3                                            50400.0           
4                                           154800.0           

[5 rows x 44 columns]

In [38]:
# Export the TF-IDF DataFrame to an Excel file
df7.to_excel('E://REVA//Healthcare Providers Data For Anomaly Detection//Medicare1//df8_output.xlsx', index=False, engine='openpyxl')